In [40]:
all_sentences = [
    "ダーウィンの進化論は嘘です",
    "バナナはおやつに入ります",
    "地球は平面です",
    "日本では応用数学科はまだまだ少ない方です",
    "数学と数理科学の違いはなんですか",
    "散歩をするとバナナがおやつに入らないかもしれないことに気づきました",
    "地球の半径を数理的に求めることができます",
    "私はバナナが好きな数学者です",
    "数学とバナナは同じです",
    "残念ながら、地球とバナナとゴリラは同じではありません",
    "ダーウィンはバナナをおやつと考えました",
    "これ以上無意味な文章を作ることをやめませんか",
    "数理の世界は長い年月を経て進歩してきましたが、人間は長い年月を経てゴリラに近づきました",
    "ダーウィンは進化論の提唱者ですが、ダテミキオはカロリー0理論の提唱者です",
    "その理論を応用することで、バナナを用いてブラックホールを生成する方法を数学的に導くことができます",
    "ピザはその高さを0に近づけることで体積が0に近づくためカロリーは0",
    "ダーウィンはゴリラの進化元です",
    "バナナのカロリーは1本86キロカロリーです",
    "どうして地球にはピザが自生していないのですか",
    "ここまでだいたい嘘"
]

In [16]:
# CUDAの使用
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [41]:
from sentence_transformers import SentenceTransformer

# 日本語に強そうなモデルを指定
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device=device)

# 20行のセンテンスを768次元のベクトルに変換（埋め込み）
all_embeddings = model.encode(all_sentences)
all_embeddings.shape

(20, 768)

In [42]:
# 検索結果が怪しいので別のtokenizerを使う
from transformers import BertJapaneseTokenizer

# 指定したtokenizerを用いる
tokenizer_cl_tohoku = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

all_tokens_cl_tohoku = [tokenizer_cl_tohoku.tokenize(sentence) for sentence in all_sentences]
all_tokens_cl_tohoku

[['ダーウィン', 'の', '進化', '論', 'は', '嘘', 'です'],
 ['バナナ', 'は', 'お', 'やつ', 'に', '入り', 'ます'],
 ['地球', 'は', '平面', 'です'],
 ['日本', 'で', 'は', '応用', '数', '学科', 'は', 'まだ', '##ま', '##だ', '少ない', '方', 'です'],
 ['数学', 'と', '数理', '科学', 'の', '違い', 'は', 'なん', 'です', 'か'],
 ['散歩',
  'を',
  'する',
  'と',
  'バナナ',
  'が',
  'お',
  'やつ',
  'に',
  '入ら',
  'ない',
  'かも',
  'しれ',
  'ない',
  'こと',
  'に',
  '気づき',
  'まし',
  'た'],
 ['地球', 'の', '半径', 'を', '数理', '的', 'に', '求める', 'こと', 'が', 'でき', 'ます'],
 ['私', 'は', 'バナナ', 'が', '好き', 'な', '数学', '者', 'です'],
 ['数学', 'と', 'バナナ', 'は', '同じ', 'です'],
 ['残念',
  'ながら',
  '、',
  '地球',
  'と',
  'バナナ',
  'と',
  'ゴリ',
  '##ラ',
  'は',
  '同じ',
  'で',
  'は',
  'あり',
  'ませ',
  'ん'],
 ['ダーウィン', 'は', 'バナナ', 'を', 'お', 'やつ', 'と', '考え', 'まし', 'た'],
 ['これ',
  '以上',
  '無意',
  '##味',
  'な',
  '文章',
  'を',
  '作る',
  'こと',
  'を',
  'やめ',
  'ませ',
  'ん',
  'か'],
 ['数理',
  'の',
  '世界',
  'は',
  '長い',
  '年月',
  'を',
  '経',
  'て',
  '進歩',
  'し',
  'て',
  'き',
  'まし',
  'た',
  'が',
  '、',
  '人間',
  'は',
  '

In [43]:
# Pineconeの初期化
from pinecone import Pinecone, PodSpec
pc = Pinecone(api_key='1a0e3dd8-12dd-42cc-92ef-16e20a38cb4b')

In [30]:
# インデックスの作成
# all_tokensを格納するDBの作成
pc.create_index(
    name="semantic-search-in-japanese", 
    
    dimension=all_embeddings.shape[1], 
    metric="cosine",
    spec=PodSpec(
        environment='gcp-starter', 
        pod_type='starter'
    )
)

In [31]:
# インデックスに接続
index = pc.Index(name="semantic-search-in-japanese")

In [32]:
# データの挿入
upserts = []
for i, (embedding, tokens) in enumerate(zip(all_embeddings, all_tokens_cl_tohoku)):
    upserts.append((str(i), embedding.tolist(), {'tokens': tokens}))

index.upsert(vectors=upserts)

{'upserted_count': 20}

In [44]:
# 検索
query = "数学"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '4',
              'metadata': {'tokens': ['数学',
                                      'と',
                                      '数理',
                                      '科学',
                                      'の',
                                      '違い',
                                      'は',
                                      'なん',
                                      'です',
                                      'か']},
              'score': 0.705078781,
              'values': []},
             {'id': '6',
              'metadata': {'tokens': ['地球',
                                      'の',
                                      '半径',
                                      'を',
                                      '数理',
                                      '的',
                                      'に',
                                      '求める',
                                      'こと',
                                      'が',
           

In [45]:
# 検索
query = "地球と数学"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '6',
              'metadata': {'tokens': ['地球',
                                      'の',
                                      '半径',
                                      'を',
                                      '数理',
                                      '的',
                                      'に',
                                      '求める',
                                      'こと',
                                      'が',
                                      'でき',
                                      'ます']},
              'score': 0.763870358,
              'values': []},
             {'id': '2',
              'metadata': {'tokens': ['地球', 'は', '平面', 'です']},
              'score': 0.622525,
              'values': []},
             {'id': '4',
              'metadata': {'tokens': ['数学',
                                      'と',
                                      '数理',
                                      '科学',
                                  

In [37]:
# 検索
query = "バナナとゴリラ"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '1',
              'metadata': {'tokens': ['バナナ', 'は', 'お', 'やつ', 'に', '入り', 'ます']},
              'score': 0.911038876,
              'values': []},
             {'id': '9',
              'metadata': {'tokens': ['残念',
                                      'ながら',
                                      '、',
                                      '地球',
                                      'と',
                                      'バナナ',
                                      'と',
                                      'ゴリ',
                                      '##ラ',
                                      'は',
                                      '同じ',
                                      'で',
                                      'は',
                                      'あり',
                                      'ませ',
                                      'ん']},
              'score': 0.723820031,
              'values': []},
             {'id': '5',
             

In [47]:
# 検索
query = "バナナと平面"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '1',
              'metadata': {'tokens': ['バナナ', 'は', 'お', 'やつ', 'に', '入り', 'ます']},
              'score': 0.838205338,
              'values': []},
             {'id': '9',
              'metadata': {'tokens': ['残念',
                                      'ながら',
                                      '、',
                                      '地球',
                                      'と',
                                      'バナナ',
                                      'と',
                                      'ゴリ',
                                      '##ラ',
                                      'は',
                                      '同じ',
                                      'で',
                                      'は',
                                      'あり',
                                      'ませ',
                                      'ん']},
              'score': 0.720716298,
              'values': []},
             {'id': '8',
             

In [51]:
# 検索
query = "ダーウィンは数学者ですか？"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '16',
              'metadata': {'tokens': ['ダーウィン',
                                      'は',
                                      'ゴリ',
                                      '##ラ',
                                      'の',
                                      '進化',
                                      '元',
                                      'です']},
              'score': 0.746318281,
              'values': []},
             {'id': '13',
              'metadata': {'tokens': ['ダーウィン',
                                      'は',
                                      '進化',
                                      '論',
                                      'の',
                                      '提唱',
                                      '者',
                                      'です',
                                      'が',
                                      '、',
                                      'ダ',
                                      '##テ',
   

In [39]:
# ゴリラが検索できない
# セマンティック検索
query = "ゴリラ"
xq = model.encode([query]).tolist()

result = index.query(vector=xq, top_k=5, includeMetadata=True)
result

{'matches': [{'id': '19',
              'metadata': {'tokens': ['ここ', 'まで', 'だい', '##たい', '嘘']},
              'score': 0.527226746,
              'values': []},
             {'id': '1',
              'metadata': {'tokens': ['バナナ', 'は', 'お', 'やつ', 'に', '入り', 'ます']},
              'score': 0.415789127,
              'values': []},
             {'id': '11',
              'metadata': {'tokens': ['これ',
                                      '以上',
                                      '無意',
                                      '##味',
                                      'な',
                                      '文章',
                                      'を',
                                      '作る',
                                      'こと',
                                      'を',
                                      'やめ',
                                      'ませ',
                                      'ん',
                                      'か']},
              'score': 0.371

In [11]:
pc.delete_index("semantic-search-in-japanese")